In [2]:
! pip install python-dotenv langchain langchain-community langchainhub openai tiktoken azure-ai-documentintelligence azure-identity azure-search-documents==11.4.0b8
! pip install langchain --upgrade

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [3]:
# Defines the function to chunk and embed a document

from langchain import hub
from langchain.chat_models import AzureChatOpenAI
from langchain_community.document_loaders import AzureAIDocumentIntelligenceLoader
from langchain.embeddings import AzureOpenAIEmbeddings
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import MarkdownHeaderTextSplitter
from langchain.vectorstores.azuresearch import AzureSearch

from dotenv import load_dotenv
import os

def process_file(file_path, index_name):


    load_dotenv()  # take environment variables from .env.
    os.environ["openai.api_type"] = os.getenv("openai.api_type")
    os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
    os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
    doc_intelligence_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")
    doc_intelligence_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")
    api_version = os.getenv("openai.api_version")
    ada_deployed_model = os.getenv("ada")
    gpt4_deployed_model = os.getenv("gpt4")

    # Initiate Azure AI Document Intelligence to load the document. You can either specify file_path or url_path to load the document.
    loader = AzureAIDocumentIntelligenceLoader(
        file_path=file_path,
        api_key=doc_intelligence_key,
        api_endpoint=doc_intelligence_endpoint,
        api_model="prebuilt-layout",
    )
    docs = loader.load()

    # Split the document into chunks base on markdown headers.
    headers_to_split_on = [
        ("#", "Header 1"),
        ("##", "Header 2"),
        ("###", "Header 3"),
    ]
    text_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)

    docs_string = docs[0].page_content
    splits = text_splitter.split_text(docs_string)

    print("Length of splits: " + str(len(splits)))

    # Embed the splitted documents and insert into Azure Search vector store
    # openai.api_base = os.getenv("openai.api_base")
    import openai

    openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")

    aoai_embeddings = AzureOpenAIEmbeddings(
        azure_deployment=ada_deployed_model,
        openai_api_version=api_version,  # e.g., "2023-07-01-preview"
    )

    vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
    vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")


    vector_store: AzureSearch = AzureSearch(
        azure_search_endpoint=vector_store_address,
        azure_search_key=vector_store_password,
        index_name=index_name,
        embedding_function=aoai_embeddings.embed_query,
    )

    vector_store.add_documents(documents=splits)

    

In [4]:
# Let's create two indexes 
process_file("../sample_docs/pdf-sample.pdf", "pdf-sample")
process_file("../sample_docs/Mac.pdf", "mac")


Length of splits: 2


/Users/miguelp/Library/Python/3.9/lib/python/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.azure_openai.AzureOpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import AzureOpenAIEmbeddings`.
  warn_deprecated(


In [5]:
# Defines the function to store prompts and which index to search
# This define our corpus of knowledge , of which embedding store (vector store or db) to search

def feed_corpus(question, engine):


    load_dotenv()  # take environment variables from .env.
    os.environ["openai.api_type"] = os.getenv("openai.api_type")
    os.environ["AZURE_OPENAI_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT")
    os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
    api_version = os.getenv("openai.api_version")
    ada_deployed_model = os.getenv("ada")



    import openai

    openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")

    aoai_embeddings = AzureOpenAIEmbeddings(
        azure_deployment=ada_deployed_model,
        openai_api_version=api_version,  # e.g., "2023-07-01-preview"
    )

    vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
    vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")

    from azure.search.documents.indexes.models import (
    ScoringProfile,
    SearchableField,
    SearchField,
    SearchFieldDataType,
    SimpleField,
    TextWeights)

    fields = [
    SimpleField(
        name="id",
        type=SearchFieldDataType.String,
        key=True,
        filterable=True,
    ),
    SearchableField(
        name="content",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    SearchField(
        name="content_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        searchable=True,
        vector_search_dimensions=1536,
        vector_search_configuration="default",
    ),
    SearchableField(
        name="metadata",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    # Additional field to store which engine got an answer
    SearchableField(
        name="engine",
        type=SearchFieldDataType.String,
        searchable=True,
    ),
    # Additional field for filtering on document source
    SimpleField(
        name="source",
        type=SearchFieldDataType.String,
        filterable=True,
    ),
]


    vector_store: AzureSearch = AzureSearch(
        azure_search_endpoint=vector_store_address,
        azure_search_key=vector_store_password,
        index_name="corpus",
        embedding_function=aoai_embeddings.embed_query,
        fields=fields
    )

    
    vector_store.add_texts(
        [question],
        [
            { "content": question, "engine": engine}
        ]
        
        )

In [6]:
feed_corpus("Who can read a PDF file?", "pdf-sample")